In [ ]:
# 2020-10-20 created by Akson

In [ ]:
# code11.1

from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import matplotlib.cm as cm

mnist = fetch_openml('mnist_784', data_home = 'data/mnist')

# 每次加载过慢，测试时就把代码拆分了

In [ ]:
# 接上

counter = 1
for i in range(1, 4):
    for j in range(1, 6):
        plt.subplot(3, 5, counter)
        plt.imshow(mnist.data[(i - 1) * 8000 + j].reshape((28, 28)), cmap = cm.Greys_r)
        plt.axis('off')
        counter += 1
plt.show()

In [ ]:
# code11.2
# 使用SVM来做图片分类

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# 网格搜寻会创建额外的进程，所以代码最好是从__main__开始好一点，但jupyter-notebook好像不用
#if __name__ == '__main__':

X, y = mnist.data, mnist.target
X = X / 255.0 * 2 - 1 # 让标记中像素强度值限制到-1和1之间

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 11)

pipeline = Pipeline([
    ('clf', SVC(kernel = 'rbf', gamma = 0.01, C = 100))
])

parameters = {
    # 'clf__gamma': (0.01, 0.03, 0.1, 0.3, 1),
    # 'clf__C': (0.1, 0.3, 1, 3, 10, 30)
    
    # 为了降低我这小破电脑的压力，我简化了一下参数
    'clf__gamma': (0.01, 0.1),
    'clf__C': (1, 3)
}

grid_search = GridSearchCV(pipeline, parameters, cv = 3, n_jobs = -1, verbose = 2, scoring = 'accuracy')
grid_search.fit(X_train[: 10000], y_train[: 10000])

print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
    
predictions = grid_search.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
# code11.3
# 自然图片分类

import os
import numpy as np
from PIL import Image

X = []
y = []

# 数据集过大，可在下面这个网址自行下载，解压到项目根目录里即可
# http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/EnglishImg.tgz
for path, subdirs, files in os.walk('English/Img/GoodImg/Bmp/'):
    for subdir in subdirs:
        for path, subdirs, files in os.walk('English/Img/GoodImg/Bmp/' + subdir):
            for filename in files:
                f = os.path.join(path, filename)
                target = filename[3: filename.index('-')]
                img = Image.open(f).convert('L').resize((30, 30), resample = Image.LANCZOS)
                X.append(np.array(img).reshape(900, ))
                y.append(target)
X = np.array(X)

In [ ]:
# code11.4

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 11)
pipeline = Pipeline([
    ('clf', SVC(kernel = 'rbf', gamma = 0.01, C = 100))
])

parameters = {
    # 'clf__gamma': (0.01, 0.03, 0.1, 0.3, 1),
    # 'clf__C': (0.1, 0.3, 1, 3, 10, 30)
    
    # 为了降低我这小破电脑的压力，我简化了一下参数
    'clf__gamma': (0.01, 0.1),
    'clf__C': (1, 3)
}

grid_search = GridSearchCV(pipeline, parameters, cv = 3, n_jobs = -1, verbose = 2, scoring = 'accuracy')
grid_search.fit(X_train[: 10000], y_train[: 10000])

print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))
    
predictions = grid_search.predict(X_test)
print(classification_report(y_test, predictions))